In [ ]:
import ee
#ee.Authenticate()
ee.Initialize()

In [ ]:
#!pip install geemap

In [ ]:
import geemap
Map = geemap.Map(center=[40,-100], zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
# Add Earth Engine dataset
Map.setCenter(-110, 40, 5)
fc = ee.FeatureCollection('TIGER/2018/States').filter(ee.Filter.eq('STUSPS', 'MN'))

# Create a Landsat 7, median-pixel composite for Spring of 2000.
collection = ee.ImageCollection('LE7_L1T').filterDate("2000-05-01", "2000-10-31") \
      .filterBounds(fc)
image1 = collection.median()
# Map.addLayer(image1)

# # Clip to the output image to the California state boundary.
# # fc = (ee.FeatureCollection('ft:1fRY18cjsHzDgGiJiS2nnpUU3v9JPDc2HNaR7Xk8')
# #       .filter(ee.Filter().eq('Name', 'Minnesota')))
image2 = image1.clipToCollection(fc)
# Select the red, green and blue bands.
image = image2.select('B4', 'B3', 'B2')
Map.addLayer(image, {'gain': [1.4, 1.4, 1.1]}, 'Landsat 7')

In [ ]:
print(image.getInfo()['bands'][0])

{'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}


In [ ]:
#create a monthly mosaic using cloud-less images

p1 = ee.Geometry.Point([103.521, 13.028])
p2 = ee.Geometry.Point([105.622, 13.050])
Date_Start = ee.Date('2000-05-01')
Date_End = ee.Date('2007-12-01')
Date_window = ee.Number(30)

# Create list of dates for time series
n_months = Date_End.difference(Date_Start, 'month').round()
print("Number of months:", n_months.getInfo())
dates = ee.List.sequence(0, n_months, 1)
print(dates.getInfo())

def make_datelist(n):
    return Date_Start.advance(n, 'month')


dates = dates.map(make_datelist)
print(dates.getInfo())


def fnc(d1):
    S1 = ee.ImageCollection('LANDSAT/LT5_L1T_TOA') \
        .filterDate('2000-05-01', '2007-12-01') \
        .filter(ee.Filter.calendarRange(1, 14, 'month')) \
        .sort('CLOUD_COVER') \
        .filterBounds(p1).first()
    S2 = ee.ImageCollection('LANDSAT/LT5_L1T_TOA') \
        .filterDate('2000-05-01', '2007-12-01') \
        .filter(ee.Filter.calendarRange(1, 14, 'month')) \
        .sort('CLOUD_COVER') \
        .filterBounds(p2).first()

    mosaic = ee.ImageCollection([ee.Image(S1), ee.Image(S2)]).mosaic()

    return mosaic


list_of_images = dates.map(fnc)
print('list_of_images', list_of_images.getInfo())
mt = ee.ImageCollection(list_of_images)
print(mt.getInfo())

Number of months: 91
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91]
[{'type': 'Date', 'value': 957139200000}, {'type': 'Date', 'value': 959817600000}, {'type': 'Date', 'value': 962409600000}, {'type': 'Date', 'value': 965088000000}, {'type': 'Date', 'value': 967766400000}, {'type': 'Date', 'value': 970358400000}, {'type': 'Date', 'value': 973036800000}, {'type': 'Date', 'value': 975628800000}, {'type': 'Date', 'value': 978307200000}, {'type': 'Date', 'value': 980985600000}, {'type': 'Date', 'value': 983404800000}, {'type': 'Date', 'value': 986083200000}, {'type': 'Date', 'value': 988675200000}, {'type': 'Date', 'value': 991353600000}, {'type': 'Date', 'value': 993945600000}, {'ty

In [ ]:
Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.
Map.addLayer(mt,{},"mt")
Map.setCenter(103.521, 13.028, 5)